# Questioning the Effect of Physiological Heartbeat Synchrony in Romantic Dyads. A Preregistered Deep Learning Analysis.

In [74]:
# For Google Colab only:
from google.colab import drive
drive.mount('/content/drive/MyDrive/Masterarbeit/Code/two-hearts/')
google = "/content/drive/MyDrive/Masterarbeit/Code/two-hearts/"

import sys
sys.path.append(google)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [54]:
# Import libraries
import os
import datetime

import IPython
import IPython.display
import random
import numpy as np
from numpy import array, hstack
import pandas as pd
import matplotlib.pyplot as plt
import pickle as pkl
import pickle
from tensorflow.python.keras.layers import deserialize, serialize
from tensorflow.python.keras.saving import saving_utils

import tensorflow.keras
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import LSTM, Dense, RepeatVector, TimeDistributed, Input, BatchNormalization, multiply, concatenate, Flatten, Activation, dot
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping
import pydot as pyd
from tensorflow.keras.utils import plot_model, model_to_dot
tensorflow.keras.utils.pydot = pyd

from lists import list_str

print("TensorFlow version:",tensorflow.version.VERSION)

# Hotfix for tensorflow < 2.6.0 to make keras models pickable
# from https://github.com/tensorflow/tensorflow/issues/34697#issuecomment-627193883

def unpack(model, training_config, weights):
    restored_model = deserialize(model)
    if training_config is not None:
        restored_model.compile(
            **saving_utils.compile_args_from_training_config(
                training_config
            )
        )
    restored_model.set_weights(weights)
    return restored_model

# Hotfix function
def make_keras_picklable():

    def __reduce__(self):
        model_metadata = saving_utils.model_metadata(self)
        training_config = model_metadata.get("training_config", None)
        model = serialize(self)
        weights = self.get_weights()
        return (unpack, (model, training_config, weights))

    cls = Model
    cls.__reduce__ = __reduce__

# Run the function
ver = tensorflow.version.VERSION
if float(ver[:-2]) < 2.6: 
    make_keras_picklable()
    print("Hotfix applied")

TensorFlow version: 2.3.0
Hotfix applied


## Deep Learning

In [56]:
# Set sampling rate
sampling_rate = 50
print("Sampling rate:", sampling_rate)

# Set number of time steps
n_steps_in, n_steps_out = 5*sampling_rate, 2*sampling_rate
print("Time steps:", n_steps_in, n_steps_out)

condition = ["sit","gaze","gaze_swap"]
print("Conditions:", condition)

print("Participants:",list_str)

Sampling rate: 50
Time steps: 250 100
Conditions: ['sit', 'gaze', 'gaze_swap']
Participants: ['01', '02', '03', '04']


In [4]:
# Split a multivariate sequence into samples (modified from Brownlee 2018, p.156)
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        if i % (sampling_rate) == 0: # to remove redundancy in samples
            # find the end of this pattern
            end_ix = i + n_steps_in
            out_end_ix = end_ix + n_steps_out
            # check if we are beyond the dataset
            if out_end_ix > len(sequences):
                break
            # gather input and output parts of the pattern
            seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:out_end_ix, :]
            X.append(seq_x)
            y.append(seq_y)
    return array(X), array(y)

In [5]:
# Prepare sample data
def sample_preperation(condition): #google_colab = False
    # Load data
    data = np.load("data/data_"+condition+".npy")
    print("Loaded data with shape", data.shape,"and type",data.dtype)

    # Create samples
    X_input_train = X_input_vali = X_input_test = np.empty((0, n_steps_in, 2))
    y_output_train = y_output_vali = y_output_test = np.empty((0, n_steps_out, 2))

    idx = list(range(len(list_str)))[::2] # for all dyads
    # idx = [0] # for testing with 1 dyad only

    for i in range(len(idx)):
        # define input sequence
        in_seq1 = data[idx[i]]
        in_seq2 = data[idx[i]+1]
        # convert to [rows, columns] structure
        in_seq1 = in_seq1.reshape((len(in_seq1), 1))
        in_seq2 = in_seq2.reshape((len(in_seq2), 1))
        # horizontally stack columns
        dataset = hstack((in_seq1, in_seq2))
        # covert into input/output
        X, y = split_sequences(dataset, n_steps_in, n_steps_out)
        # data split
        num_train_samples = int(0.6 * len(X))
        num_val_samples = int(0.2 * len(X))
        num_test_samples = len(X) - num_train_samples - num_val_samples
        # append data for multiple dyads
        X_input_train = np.append(X_input_train,X[:num_train_samples],axis=0)
        y_output_train = np.append(y_output_train,y[:num_train_samples],axis=0)
        X_input_vali = np.append(X_input_vali,X[num_train_samples:(num_train_samples+num_val_samples)],axis=0)
        y_output_vali = np.append(y_output_vali,y[num_train_samples:(num_train_samples+num_val_samples)],axis=0)
        X_input_test = np.append(X_input_test,X[(num_train_samples+num_val_samples):],axis=0)
        y_output_test = np.append(y_output_test,y[(num_train_samples+num_val_samples):],axis=0)
    
    # Create dictionary
    samples = {
        "X_input_train": X_input_train,
        "y_output_train": y_output_train,
        "X_input_vali": X_input_vali,
        "y_output_vali": y_output_vali,
        "X_input_test": X_input_test,
        "y_output_test": y_output_test
    }

    ## Plot data
    # fig = plt.figure(figsize=(6,1), dpi=96)
    # example = np.append(X_input_train[14,:,0], y_output_train[14,:,0])
    # example2 = np.append(X_input_train[14,:,1], y_output_train[14,:,1])
    # plt.plot(example)
    # plt.plot(example2)

    print("Number of dyads:", max(idx))
    print("num_train_samples per dyad:", num_train_samples)
    print("num_val_samples per dyad:", num_val_samples)
    print("num_test_samples per dyad:", num_test_samples)
    print("Length of samples for each set:", len(X_input_train), len(X_input_vali), len(X_input_test))

    return samples

In [6]:
# Define simple seq2seq model 
# Modified from Wieniawska 2020 
# (https://levelup.gitconnected.com/building-seq2seq-lstm-with-luong-attention-in-keras-for-time-series-forecasting-1ee00958decb)

def lstm_decoder_encoder(samples, n_hidden = 100):
    # Input layer
    input_train = Input(shape=(samples["X_input_train"].shape[1], samples["X_input_train"].shape[2]))
    output_train = Input(shape=(samples["y_output_train"].shape[1], samples["y_output_train"].shape[2]))
    # print(input_train)
    # print(output_train)

    # Encoder LSTM with state_h and state_c
    encoder_last_h1, encoder_last_h2, encoder_last_c = LSTM(
    n_hidden, activation='elu', dropout=0.2, recurrent_dropout=0.2, 
    return_sequences=False, return_state=True)(input_train)
    # print(encoder_last_h1)
    # print(encoder_last_h2)
    # print(encoder_last_c)

    # Batch normalisation to avoid gradient explosion
    encoder_last_h1 = BatchNormalization(momentum=0.6)(encoder_last_h1)
    encoder_last_c = BatchNormalization(momentum=0.6)(encoder_last_c)

    # Decoder LSTM
    decoder = RepeatVector(output_train.shape[1])(encoder_last_h1)
    decoder = LSTM(n_hidden, activation='elu', dropout=0.2, recurrent_dropout=0.2, return_state=False, return_sequences=True)(
        decoder, initial_state=[encoder_last_h1, encoder_last_c])
    # print(decoder)

    # Dense layer with repeated weights
    out = TimeDistributed(Dense(output_train.shape[2]))(decoder)
    # print(out)

    # Compile model
    model = Model(inputs=input_train, outputs=out)
    opt = Adam(learning_rate=0.001, clipnorm=1)
    model.compile(optimizer=opt, loss='mean_squared_error', metrics=['mae'])
    model.summary()

    return model

In [7]:
# Define seq2seq model with Loung attention
# Modified from Wieniawska 2020 
# (https://levelup.gitconnected.com/building-seq2seq-lstm-with-luong-attention-in-keras-for-time-series-forecasting-1ee00958decb)

def lstm_decoder_encoder_loung_attention(samples, n_hidden = 100):
    # Input layer
    input_train = Input(shape=(samples["X_input_train.shape[1]"], samples["X_input_train.shape[2]"]))
    output_train = Input(shape=(samples["y_output_train.shape[1]"], samples["y_output_train.shape[2]"]))

    # Encoder LSTM
    encoder_stack_h, encoder_last_h, encoder_last_c = LSTM(
        n_hidden, activation='elu', dropout=0.2, recurrent_dropout=0.2, 
        return_state=True, return_sequences=True)(input_train)
    # print(encoder_stack_h)
    # print(encoder_last_h)
    # print(encoder_last_c)

    # Batch normalisation to avoid gradient explosion
    encoder_last_h = BatchNormalization(momentum=0.6)(encoder_last_h)
    encoder_last_c = BatchNormalization(momentum=0.6)(encoder_last_c)

    # Decoder LSTM
    decoder_input = RepeatVector(output_train.shape[1])(encoder_last_h)
    # print(decoder_input)

    decoder_stack_h = LSTM(n_hidden, activation='elu', dropout=0.2, recurrent_dropout=0.2,
    return_state=False, return_sequences=True)(
    decoder_input, initial_state=[encoder_last_h, encoder_last_c])
    # print(decoder_stack_h)

    # Attention layer
    attention = dot([decoder_stack_h, encoder_stack_h], axes=[2, 2])
    attention = Activation('softmax')(attention)
    # print(attention)

    # Calculate context vector with batch normalisation
    context = dot([attention, encoder_stack_h], axes=[2,1])
    context = BatchNormalization(momentum=0.6)(context)
    # print(context)

    # Combine context vector with stacked hidden states of decoder for input to the last dense layer
    decoder_combined_context = concatenate([context, decoder_stack_h])
    # print(decoder_combined_context)

    # Dense layer with repeated weights
    out = TimeDistributed(Dense(output_train.shape[2]))(decoder_combined_context)
    # print(out)

    # Compile model
    model = Model(inputs=input_train, outputs=out)
    opt = Adam(learning_rate=0.001, clipnorm=1)
    model.compile(loss='mean_squared_error', optimizer=opt, metrics=['mae'])
    model.summary()

In [8]:
# Fit model
def fit_model(model, samples):
    epc = 300
    es = EarlyStopping(monitor='val_loss', mode='min', patience=50, restore_best_weights=True)
    history = model.fit(samples["X_input_train"], samples["y_output_train"],  validation_data=(samples["X_input_vali"],samples["y_output_vali"]), 
                        epochs=epc, verbose=1, callbacks=[es], 
                        batch_size=64, shuffle=False)
    # model.save("model_forecasting_seq2seq.h5")

    return model, history

In [10]:
# Execute over all conditions for n trials
trial = ["01","02","03"]
samples_all = {}
model_all = {}
history_all = {}
results_all = {}
data_all = {}
for x in range(len(trial)): data_all[trial[x]] = {} # prepare nested dictionary

for j in range(len(trial)):
    for i in range(len(condition)):
        samples = sample_preperation(condition[i])
        samples_all[condition[i]] = samples
        model = lstm_decoder_encoder(samples)
        # model = lstm_decoder_encoder_loung_attention(samples)
        model, history = fit_model(model, samples)
        model_all[condition[i]] = model
        history_all[condition[i]] = history
        results = model.evaluate(samples["X_input_test"], samples["y_output_test"], batch_size=64)
        results_all[condition[i]] = results
    data_all[trial[j]]["samples_all"] = samples_all
    data_all[trial[j]]["model_all"] = model_all
    data_all[trial[j]]["history_all"] = history_all
    data_all[trial[j]]["results_all"] = results_all

data_all_file = open("data_all.pkl","wb")
pkl.dump(data_all,data_all_file)
data_all_file.close()


Loaded data with shape (4, 14800) and type float32
Number of dyads: 2
num_train_samples per dyad: 174
num_val_samples per dyad: 58
num_test_samples per dyad: 58
Length of samples for each set: 348 116 116
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 250, 2)]     0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 100), (None, 41200       input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 100)          400         lstm[0][0]                       
________________________________________________________________________________